In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from src.models.models import load_from_checkpoint as lfc
from src.utils.score_utils import metrics_mapping
from src.utils.generation_utils import load_model, all_tfr_decode
import matplotlib.pyplot as plt

import logging
import pandas as pd
import torch
import time
device = "cuda:1"

2023-03-05 06:45:12.232628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 06:45:12.232648: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
setting = "table2text"
# get generation model with monkey patching based on setting
# setting, TFR_DECODE, DEVICE, TRAIN
mod, tok, dset, dec_pref = load_model(setting, True, "cuda:1", False)
dset = list(dset)


/home/prasann/miniconda3/envs/rewardenv/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Created a temporary directory at /tmp/tmp5u6hda0z
Writing /tmp/tmp5u6hda0z/_remote_module_non_scriptable.py
Some weights of the model checkpoint at facebook/bart-base were not used when initializing CustomBartModel

using correct weight adjustments
using model  bart-base
FREEZE BART SHARED EMBEDS
freeze embeds


In [3]:
def weightdecay(numtoks):
    return 5/numtoks

def onlytfr(numtoks):
    return 0

#allcands = pd.read_csv("tmp.csv", index_col=0)
def rer_met(rer, tgt, df, sorter="ref"):
    N=1
    fsort = df.sort_values(by=[sorter, rer], ascending=[True, False]).groupby(sorter, as_index=False).nth[:N]
    return fsort[tgt].mean()

# get results in a df for a given configuration 
def test_config(mod, tok, data, config, more=True):
    allcands = all_tfr_decode(mod, tok, data, config)
    if more:
        allcands['modsco'] = [float(f) for f in allcands['modsco']]
        metrics_mapping("pqe", allcands)
        metrics_mapping("parent",allcands)
    return allcands

In [4]:
mod.man_pref = None

In [5]:
allcands = test_config(mod, tok, dset[10:20], {
        "max_len":386,
        "device":'cuda:1',
        "beam_size":50,
        "dec_prefix":dec_pref,
        "tfr_interv":6,
        "tfr_beams":13,
        "weightfunc":onlytfr
}, True)

Time taken :  15.484527587890625


Some weights of the model checkpoint at facebook/bart-base were not used when initializing CustomBartModel: ['final_logits_bias']
- This IS expected if you are initializing CustomBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_57/checkpoints/epoch=1-step=50000.ckpt.


using correct weight adjustments
using model  bart-base
FREEZE BART SHARED EMBEDS
freeze embeds
0.0 %
32.0 %
64.0 %
96.0 %
TOOK TIME  1.09
references loaded
0
1000
0
1000
0
1000
references processed
0
hyps processed
<module 'tqdm' from '/home/prasann/miniconda3/envs/rewardenv/lib/python3.8/site-packages/tqdm/__init__.py'>
Using 4 processes, starting now
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | fal

Computing PARENT: 100%|███████████████████████████████████████████████| 500/500 [00:00<00:00, 2484.14it/s]


In [6]:
rer_met("pqe", "precision", allcands, "ref")

0.6521902073342654

In [22]:
def get_pref_extensions(mod, tok, data, prefs, bsize, preflen):
    config = {
            "max_len":preflen,
            "device":'cuda:1',
            "beam_size":prefs,
            "dec_prefix":dec_pref,
            "tfr_interv":1000,
            "tfr_beams":13,
            "weightfunc":onlytfr
    }
    mod.man_pref = None
    prefs = all_tfr_decode(mod, tok, data, config).reset_index()
    config['beam_size']=bsize
    config['max_len']=384
    cdfs = []
    for i in range(len(prefs)):
        dat = [[prefs['src'][i], prefs['ref'][i]]]
        print(dat)
        print("<pad> "+str(prefs['hyp'][i]).strip())
        forced = tok("<pad> "+str(prefs['hyp'][i]).strip(), return_tensors="pt").input_ids.to(device)[:, :-1][0]
        mod.man_pref = forced
        cdf = all_tfr_decode(mod, tok, dat, config)
        cdf['prefix']=str(prefs['hyp'][i])
        cdfs.append(cdf)
    return pd.concat(cdfs).reset_index().drop(columns=['index'])

In [18]:
dset[19][0]

'<H> Abner W. Sibal <R> military Branch <T> United States Army <H> United States Army <R> battles <T> Korean War'

In [23]:
bconfs = [50]
bprefs = [4, 8, 12]
for p in bprefs:
    for b in bconfs:
        pexts = get_pref_extensions(mod, tok, dset[10:20], 50, b, p)
        pexts['modsco'] = [float(f) for f in pexts['modsco']]

        metrics_mapping("pqe", pexts)
        metrics_mapping("parent",pexts)

        pexts.to_csv("output/prefexp50_"+str(b)+"_"+str(p)+".csv")

94.08 %
94.72 %
95.36 %
96.0 %
96.64 %
97.28 %
97.92 %
98.56 %
99.2 %
99.83999999999999 %
TOOK TIME  692.84
references loaded
0
1000
0
1000
0
1000
references processed
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
hyps processed
<module 'tqdm' from '/home/prasann/miniconda3/envs/latclone/lib/python3.8/site-packages/tqdm/__init__.py'>
Using 4 processes, starting now
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` be

Computing PARENT: 100%|██████████████████████████████████████████████████████| 25000/25000 [00:24<00:00, 1011.20it/s]


Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Time taken :  4.942878246307373
[['<H> Alfredo Zitarrosa <R> genre <T> Zamba artform <H> Alfredo Zitarrosa <R> background <T> solo singer', "Alfredo Zitarrosa's genre is Zamba which is considered an art form and he started out as a solo singer."]]
<pad> Alfredo Zitarrosa is a Zamba artist
Using TFR Rerank Beam Search
Time taken :  0.337144136428833
[['<H> Alfredo Zitarrosa <R> genre <T> Zamba artform <H> Alfredo Zitarrosa <R> background <T> solo singer', "Alfredo Zitarrosa's genre is Zamba which is considered an art form and he started out as a solo singer."]]
<pad> Alfredo Zitarrosa is a Zamba art
Using TFR Rerank Beam Search
Time taken :  0.405179500579834
[['<H> Alfredo Zitarrosa <R> genre <T> Zamb

Some weights of the model checkpoint at facebook/bart-base were not used when initializing CustomBartModel: ['final_logits_bias']
- This IS expected if you are initializing CustomBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_57/checkpoints/epoch=1-step=50000.ckpt.


using correct weight adjustments
using model  bart-base
FREEZE BART SHARED EMBEDS
freeze embeds
0.0 %
0.64 %
1.28 %
1.92 %
2.56 %
3.2 %
3.84 %
4.4799999999999995 %
5.12 %
5.76 %
6.4 %
7.04 %
7.68 %
8.32 %
8.959999999999999 %
9.6 %
10.24 %
10.879999999999999 %
11.52 %
12.16 %
12.8 %
13.44 %
14.08 %
14.719999999999999 %
15.36 %
16.0 %
16.64 %
17.28 %
17.919999999999998 %
18.56 %
19.2 %
19.84 %
20.48 %
21.12 %
21.759999999999998 %
22.400000000000002 %
23.04 %
23.68 %
24.32 %
24.959999999999997 %
25.6 %
26.240000000000002 %
26.88 %
27.52 %
28.16 %
28.799999999999997 %
29.439999999999998 %
30.080000000000002 %
30.72 %
31.36 %
32.0 %
32.64 %
33.28 %
33.92 %
34.56 %
35.199999999999996 %
35.839999999999996 %
36.480000000000004 %
37.12 %
37.76 %
38.4 %
39.04 %
39.68 %
40.32 %
40.96 %
41.6 %
42.24 %
42.88 %
43.519999999999996 %
44.16 %
44.800000000000004 %
45.440000000000005 %
46.08 %
46.72 %
47.36 %
48.0 %
48.64 %
49.28 %
49.919999999999995 %
50.56 %
51.2 %
51.839999999999996 %
52.4800000000000

Computing PARENT: 100%|███████████████████████████████████████████████████████| 25000/25000 [00:30<00:00, 813.19it/s]


In [105]:
pexts

,ref,hyp,src,modsco,prefix,pqe,precision,f1,recall
0,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist. He is a s...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.135869,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.540492,0.410548,0.236970,0.166552
1,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist and a solo...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.147352,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.554367,0.528097,0.573037,0.626336
2,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist and a solo...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.192266,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.541070,0.528097,0.573037,0.626336
3,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artform and solo ...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.165611,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.562612,0.598077,0.274409,0.178051
4,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artform and solo ...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.173330,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.572006,0.612498,0.275900,0.178051
...,...,...,...,...,...,...,...,...,...
295,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a soldier in the United Stat...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.145766,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.596681,0.509350,0.553214,0.605346
296,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a soldier in the United Stat...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.177756,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.623429,0.569100,0.586664,0.605346
297,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a US Army soldier who fought...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.103585,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.574392,0.524814,0.493362,0.467942
298,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a US Army soldier who served...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.104855,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.556106,0.541959,0.524748,0.537882


In [93]:
pexts = get_pref_extensions(mod, tok, dset[10:20], 10, 3, 4)
pexts['modsco'] = [float(f) for f in pexts['modsco']]

Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Using TFR Rerank Beam Search
Time taken :  1.4566121101379395
[['<H> Alfredo Zitarrosa <R> genre <T> Zamba artform <H> Alfredo Zitarrosa <R> background <T> solo singer', "Alfredo Zitarrosa's genre is Zamba which is considered an art form and he started out as a solo singer."]]
<pad> Alfredo Zitar
Using TFR Rerank Beam Search
Time taken :  0.3682889938354492
[['<H> Alfredo Zitarrosa <R> genre <T> Zamba artform <H> Alfredo Zitarrosa <R> background <T> solo singer', "Alfredo Zitarrosa's genre is Zamba which is considered an art form and he started out as a solo singer."]]
<pad> Zamba is the
Using TFR Rerank Beam Search
Time taken :  0.3612034320831299
[['<H> Alfredo Zitarrosa <R> genre <T> Zamba artform <H> Alfredo Zitarrosa <R> bac

In [94]:
metrics_mapping("pqe", pexts)
metrics_mapping("parent",pexts)

Some weights of the model checkpoint at facebook/bart-base were not used when initializing CustomBartModel: ['final_logits_bias']
- This IS expected if you are initializing CustomBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_57/checkpoints/epoch=1-step=50000.ckpt.


using correct weight adjustments
using model  bart-base
FREEZE BART SHARED EMBEDS
freeze embeds
0.0 %
53.333333333333336 %
TOOK TIME  11.24
references loaded
0
1000
0
1000
0
1000
references processed
0
hyps processed
<module 'tqdm' from '/home/prasann/miniconda3/envs/latclone/lib/python3.8/site-packages/tqdm/__init__.py'>
Using 4 processes, starting now
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | fal

Computing PARENT: 100%|█████████████████████████████████████████████████████| 300/300 [00:00<00:00, 686.49it/s]


In [102]:
pexts

,ref,hyp,src,modsco,prefix,pqe,precision,f1,recall
0,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist. He is a s...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.190892,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.540492,0.410548,0.236970,0.166552
1,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist and a solo...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.202375,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.554367,0.528097,0.573037,0.626336
2,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist and a solo...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.247289,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.541070,0.528097,0.573037,0.626336
3,Alfredo Zitarrosa's genre is Zamba which is co...,Zamba is the artform of Alfredo Zitarrosa who ...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.162357,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.547895,0.528097,0.238698,0.154197
4,Alfredo Zitarrosa's genre is Zamba which is co...,"Zamba is the artform of Alfredo Zitarrosa, who...",<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.204498,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,0.499959,0.444075,0.069477,0.037687
...,...,...,...,...,...,...,...,...,...
295,Abner W. Sibal serve in the United States army...,The US Army fought in the Korean War and is th...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.289614,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.513992,0.470017,0.478533,0.487362
296,Abner W. Sibal serve in the United States army...,The US Army fought battles in the Korean War a...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.295417,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.444522,0.328767,0.304449,0.292028
297,Abner W. Sibal serve in the United States army...,United States Army is the military branch of A...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.200434,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.660211,0.497638,0.212286,0.134921
298,Abner W. Sibal serve in the United States army...,United States Army is the military branch of t...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.216090,<H> Abner W. Sibal <R> military Branch <T> Uni...,0.647867,0.481937,0.210821,0.134921


In [95]:
pexts.to_csv("output/prefexp10_3_4.csv")

In [29]:
mod.man_pref = None
allcands = test_config(mod, tok, dset[100:110], {
        "max_len":100,
        "device":'cuda:1',
        "beam_size":50,
        "dec_prefix":dec_pref,
        "tfr_interv":1000,
        "tfr_beams":13,
        "weightfunc":onlytfr
}, False)

Time taken :  7.424964666366577


In [113]:
prefcsv.groupby("prefix", as_index=False).mean()

/tmp/ipykernel_3049731/29544730.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  prefcsv.groupby("prefix", as_index=False).mean()


,prefix,Unnamed: 0,modsco,pqe,precision,f1,recall
0,107 Camilla has an absolute Magnitude of 7.08 ...,166.0,-0.201051,0.544665,0.367771,0.054891,0.029730
1,A Wizard of Mars,31.0,-0.209454,0.740647,0.809669,0.811332,0.836093
2,"Aarhus, Denmark is",199.0,-0.110211,0.837255,0.821726,0.620841,0.499297
3,Aaron Turner is an electric guitar player who ...,128.5,-0.356729,0.603210,0.513477,0.089745,0.049262
4,Aaron Turner plays Black metal and Death metal. H,121.0,-0.307517,0.589728,0.582136,0.353727,0.279692
...,...,...,...,...,...,...,...
93,Zamba artform is,13.0,-0.261296,0.489836,0.513484,0.189883,0.151582
94,Zamba is,25.0,-0.204515,0.523001,0.502470,0.179128,0.112571
95,Zamba is Alfredo Zitarrosa's artform,16.0,-0.182609,0.474253,0.512516,0.208778,0.131201
96,Zamba is an art,22.0,-0.241037,0.424065,0.395304,0.095857,0.057543


In [4]:
def get_overlap(a, b):
    for i in range(min(len(b), len(a))):
        if a[i]!=b[i]:
            return a[:i]
    return a[:min(len(b), len(a))]

In [5]:
bconfs = [3, 10, 50]
plens = [4, 8, 12]
#bconfs = [3]
#plens = [4]

for b in bconfs:
    for p in plens:
        prefcsv = pd.read_csv('output/pref_explore/prefexp10_'+str(b)+"_"+str(p)+".csv")
        hyps = prefcsv['hyp']
        prefs = []
        for c in range(int(len(hyps)/b)):
            prefix = get_overlap(hyps[c*b], hyps[(c*b)+1])
            prefs.extend([prefix]*b)
        prefcsv['prefix']=prefs
        prefcsv.to_csv('output/pref_explore/prefexp10_'+str(b)+"_"+str(p)+".csv")

In [39]:
bconfs = [50]
plens = [4, 8, 12]

for b in bconfs:
    for p in plens:
        prefcsv = pd.read_csv('output/pref_explore/prefexp50_'+str(b)+"_"+str(p)+".csv")
        prefcsv['weight'] = prefcsv['modsco']*.3+prefcsv['pqe']
        print("beams ", b, " PREF LEN", p)
        print(rer_met('precision', 'pqe', prefcsv, "ref"))

beams  50  PREF LEN 4
0.6651833832263947
beams  50  PREF LEN 8
0.6670988440513611
beams  50  PREF LEN 12
0.6394900739192962


In [6]:
prefcsv

,Unnamed: 0,ref,hyp,src,modsco,prefix,pqe,precision,f1,recall
0,0,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist. He is a s...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.135869,Alfredo Zitarrosa is a Zamba artist,0.540492,0.410548,0.236970,0.166552
1,1,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist and a solo...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.147352,Alfredo Zitarrosa is a Zamba artist,0.554367,0.528097,0.573037,0.626336
2,2,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist and a solo...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.192266,Alfredo Zitarrosa is a Zamba artist,0.541070,0.528097,0.573037,0.626336
3,3,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist. He has pe...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.201345,Alfredo Zitarrosa is a Zamba artist,0.421954,0.301372,0.062007,0.034559
4,4,Alfredo Zitarrosa's genre is Zamba which is co...,Alfredo Zitarrosa is a Zamba artist who has pe...,<H> Alfredo Zitarrosa <R> genre <T> Zamba artf...,-0.205439,Alfredo Zitarrosa is a Zamba artist,0.476286,0.367206,0.189398,0.129004
...,...,...,...,...,...,...,...,...,...,...
4995,4995,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a US Army soldier. The US Ar...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.318523,Abner W. Sibal is a US Army soldier who,0.484199,0.392815,0.153941,0.097558
4996,4996,Abner W. Sibal serve in the United States army...,"Abner W. Sibal is a US Army soldier, who fough...",<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.320073,Abner W. Sibal is a US Army soldier who,0.513302,0.454346,0.383461,0.331709
4997,4997,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a US Army soldier that serve...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.320260,Abner W. Sibal is a US Army soldier who,0.554270,0.541959,0.524748,0.537882
4998,4998,Abner W. Sibal serve in the United States army...,Abner W. Sibal is a US Army soldier who served...,<H> Abner W. Sibal <R> military Branch <T> Uni...,-0.320460,Abner W. Sibal is a US Army soldier who,0.557912,0.448539,0.458966,0.485075


In [53]:
allcands['ref'][250]

'Bakso is a dish from the country of Indonesia (where the leader is Jusuf Kalla), it includes Tapioca.'

In [41]:
allcands['ref'][250]

'Bakso is a dish from the country of Indonesia (where the leader is Jusuf Kalla), it includes Tapioca.'

In [45]:
allcands['hyp'][250]

'Tapioca is an ingredient in Bakso which is a dish from Indonesia.'

In [58]:
chat_gptouts = [
    {
        'src':"<H> 109 Felicitas <R> discoverer <T> Christian Heinrich Friedrich Peters",
        'hyp':"The object 109 Felicitas was discovered by Christian Heinrich Friedrich Peters.",
        'ref':'Christian Heinrich Friedrich Peters was the discoverer of 109 Felicitas.'
    },
    {
        'src':"<H> Al Asad Airbase <R> operating Organisation <T> United States Air Force <H> United States Air Force <R> aircraft Fighter <T> McDonnell Douglas F-15 Eagle <H> United States Air Force <R> battles <T> 1986 United States bombing of Libya",
        'hyp':"The Al Asad Airbase is operated by the United States Air Force. The US Air Force uses aircraft such as the McDonnell Douglas F-15 Eagle. They were involved in battles like the 1986 United States bombing of Libya.",
        'ref': 'Al Asad Airbase is operated by the United States Air Force which carried out the 1986 United States bombing of Libya and which uses an aircraft fighter called the McDonnell Douglas F-15 Eagle.'
    },
    {
        'src':'<H> Bhajji <R> country <T> India <H> India <R> demonym <T> Indian people <H> India <R> leader Name <T> T. S. Thakur <H> India <R> leader Name <T> Sumitra Mahajan <H> Bhajji <R> region <T> Karnataka',
        'ref':'Indian people enjoy the dish bhajji, which originates from the Karnataka region of India. This nation is led by T.S. Thakur and Sumitra Mahajan.',
        'hyp':"Bhajji is a region located in the country of India. The people of India are referred to as Indian people. The leader of India has been T. S. Thakur and Sumitra Mahajan. Bhajji is located in the state of Karnataka.",
    },
    {
        'src':'<H> Indonesia <R> leader Name <T> Jusuf Kalla <H> Bakso <R> region <T> Indonesia <H> Bakso <R> ingredient <T> Tapioca <H> Bakso <R> country <T> Indonesia',
        'hyp':"The leader of Indonesia is Jusuf Kalla. Bakso is a dish that is popular in Indonesia. It is made using ingredients such as tapioca. Bakso originates from Indonesia.",
        'ref':'Bakso is a dish from the country of Indonesia (where the leader is Jusuf Kalla), it includes Tapioca.'
    }
]
chat_gptouts = pd.DataFrame(chat_gptouts)


In [59]:
#metrics_mapping("pqe", chat_gptouts)
metrics_mapping("parent",chat_gptouts)

references loaded
0
1000
0
1000
0
1000
references processed
0
hyps processed
<module 'tqdm' from '/home/prasann/miniconda3/envs/latclone/lib/python3.8/site-packages/tqdm/__init__.py'>
Using 4 processes, starting now
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid dea

Computing PARENT: 100%|█████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 185.90it/s]


In [61]:
metrics_mapping("pqe", chat_gptouts)

Some weights of the model checkpoint at facebook/bart-base were not used when initializing CustomBartModel: ['final_logits_bias']
- This IS expected if you are initializing CustomBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_57/checkpoints/epoch=1-step=50000.ckpt.


using correct weight adjustments
using model  bart-base
FREEZE BART SHARED EMBEDS
freeze embeds
0.0 %
TOOK TIME  0.02


In [62]:
chat_gptouts

,src,hyp,ref,precision,f1,recall,pqe
0,<H> 109 Felicitas <R> discoverer <T> Christian...,The object 109 Felicitas was discovered by Chr...,Christian Heinrich Friedrich Peters was the di...,0.871084,0.931101,1.000000,0.693575
1,<H> Al Asad Airbase <R> operating Organisation...,The Al Asad Airbase is operated by the United ...,Al Asad Airbase is operated by the United Stat...,0.618707,0.741861,0.926229,0.614694
2,<H> Bhajji <R> country <T> India <H> India <R>...,Bhajji is a region located in the country of I...,"Indian people enjoy the dish bhajji, which ori...",0.382440,0.499253,0.718805,0.546303
3,<H> Indonesia <R> leader Name <T> Jusuf Kalla ...,The leader of Indonesia is Jusuf Kalla. Bakso ...,Bakso is a dish from the country of Indonesia ...,0.346049,0.466009,0.713268,0.399429


In [25]:
pexts = pexts.reset_index().drop(columns=['index'])

In [30]:
pexts['precision']

0      1.000000
1      0.774403
2      0.757740
3      0.765440
4      0.845959
         ...   
995    0.885700
996    0.678650
997    0.708340
998    0.728056
999    0.749605
Name: precision, Length: 1000, dtype: float64

In [49]:
precs = list(pexts['precision'])
means = []
mins = []
maxes = []
for p in range(int(len(precs)/50)):
    minidist = []
    minidist = precs[p*50:(p+1)*50]
    
    means.append(sum(minidist)/len(minidist))
    mins.append(min(minidist))
    maxes.append(max(minidist))

In [60]:
pexts.loc[998]['src']

'<H> Asher and Mary Isabelle Richardson House <R> Reference Number in the National Register of Historic Places <T> 88002539'

In [72]:
pexts.loc[8]['ref']

'Asher and Mary Isabelle Richardson House has the reference number 88002539 in the National Register of Historic Places.'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = "cuda:1"

max_length = 512

hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name).to(device)
model.eval()

In [3]:
premise = ["Two women are embracing in the vicinity of a deli.", "Two women are embracing in the vicinity of a deli."]
hypothesis = ["The women are hugging outside a deli.", "The women are hugging outside a deli."]

def get_batch_entailprobs(srcs, hyps):
    tokenized_input_seq_pair = tokenizer.batch_encode_plus(list(zip(srcs, hyps)),
                                                     max_length=max_length,
                                                     return_token_type_ids=True, truncation=True, padding=True)


    input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().to("cuda:1")
    print(input_ids.shape)


    # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
    token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().to("cuda:1")
    attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().to("cuda:1")

    outputs = model(input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None)
    # Note:
    # "id2label": {
    #     "0": "entailment",
    #     "1": "neutral",
    #     "2": "contradiction"
    # },

    predicted_probability = torch.softmax(outputs[0], dim=1)
    return predicted_probability[:, 0].tolist()

In [4]:
import math

In [9]:
with torch.no_grad():
    probs = []
    bsize = 8
    iters = int(math.ceil(len(use)/bsize))
    for i in range(iters):
        probs.extend(get_batch_entailprobs(use['src'][i*bsize:(i+1)*bsize], use['hyp'][i*bsize:(i+1)*bsize]))

torch.Size([8, 222])
torch.Size([8, 223])
torch.Size([8, 225])
torch.Size([8, 225])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 280])
torch.Size([8, 284])
torch.Size([8, 284])
torch.Size([8, 284])
torch.Size([8, 285])
torch.Size([8, 284])
torch.Size([8, 285])
torch.Size([8, 285])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8, 150])
torch.Size([8, 150])
torch.Size([8, 150])
torch.Size([8, 150])
torch.Size([8, 206])
torch.Size([8, 207])
torch.Size([8, 207])
torch.Size([8, 207])
torch.Size([8, 494])
torch.Size([8, 492])
torch.Size([8, 494])
torch.Size([8, 494])
torch.Size([8, 512])
torch.Size([1, 512])


In [10]:
use['anli'] = probs

In [12]:
use['anli'][0:32]

0     0.999883
1     0.999878
2     0.999808
3     0.999904
4     0.999882
5     0.999894
6     0.999876
7     0.999880
8     0.999895
9     0.999882
10    0.999792
11    0.999892
12    0.999856
13    0.999849
14    0.999923
15    0.999775
16    0.999892
17    0.999792
18    0.999911
19    0.999903
20    0.999880
21    0.999904
22    0.997477
23    0.998264
24    0.999817
25    0.999166
26    0.999894
27    0.999836
28    0.999863
29    0.999579
30    0.999895
31    0.999582
Name: anli, dtype: float64

In [8]:
use = pd.read_csv("UniEval/minitest.csv")

In [53]:
use['ref'][0]

'The fast food chain KFC is to sue three Chinese firms it accuses of using social media to spread false rumours about its food, including that it used eight-legged chickens.'

In [16]:
ind = 1
list(use['anli'][ind*32:(ind+1)*32])

[0.9981391429901123,
 0.9983691573143005,
 0.9955908060073853,
 0.99381422996521,
 0.9951262474060059,
 0.9981141090393066,
 0.9948915243148804,
 0.9989056587219238,
 0.9970287680625916,
 0.998856782913208,
 0.9969013929367065,
 0.9992901086807251,
 0.9976898431777954,
 0.9994327425956726,
 0.9983662962913513,
 0.9973734617233276,
 0.997751772403717,
 0.9973438382148743,
 0.9912968873977661,
 0.9974568486213684,
 0.9976832866668701,
 0.9972801208496094,
 0.9987666606903076,
 0.9982155561447144,
 0.998895525932312,
 0.9987063407897949,
 0.9996168613433838,
 0.9986095428466797,
 0.99875807762146,
 0.9970184564590454,
 0.9951724410057068,
 0.9979181885719299]

In [33]:
below = use[use['anli']<.5]
below = below.reset_index()

In [29]:
ind = 8
print(below['hyp'].iloc[ind])
print(below['ref'].iloc[ind])


Derbyshire's Lord's cricket ground has played host to some of the world's best players over the years.
The Oval this week stages its 100th Test when England face South Africa.


In [37]:
below.iloc[74:45+32]

,index,Unnamed: 0,src,ref,hyp,coherence,anli
74,931,931,"Lieutenant Peter Coughlin, 34, had been drinki...",A married Royal Navy officer has been found gu...,A Royal Navy officer has been found guilty of ...,0.958726,0.098419
75,935,935,"Lieutenant Peter Coughlin, 34, had been drinki...",A married Royal Navy officer has been found gu...,A Royal Navy officer has been found guilty of ...,0.963018,0.025592
76,938,938,"Lieutenant Peter Coughlin, 34, had been drinki...",A married Royal Navy officer has been found gu...,A Royal Navy officer has been found guilty of ...,0.964962,0.034778


In [17]:
print("Premise:", premise)
print("Hypothesis:", hypothesis)
print("Entailment:", predicted_probability[0])
print("Neutral:", predicted_probability[1])
print("Contradiction:", predicted_probability[2])

Premise: ['Two women are embracing in the vicinity of a deli.', 'Two women are embracing in the vicinity of a deli.']
Hypothesis: ['The women are hugging outside a deli.', 'The women are hugging outside a deli.']


NameError: name 'predicted_probability' is not defined